## 1D convolution (implementation should  not directly use model.fit()

1. Use linear discrete 1D convolution to create a database with at least 1000 entries.  Each entry contains one of five "sounds". For each entry, linearly convolve a new instance of 2000 identical, independently distributed Gaussian samples with one of 5 different filters(impulse response, kernels). If you play the files over a loudspeaker (8kHz sampling) they should sound different.  [One method: Take a Hamming window shape (readily available) of suitable length (short is recommended) as a first kernel (filter). Then multiply said kernel by sin(wt) where t is time, with different frequencies w between 0 and pi, to create 4 more filters, each passing through a different frequency band. 
2. Design a simple 1 dimensional convolutional network using convolution, pooling and other tools with a one-hot output that classifies the sounds in the file. 
3. Test and optimise your setup with suitable methods. 
4. Extend this toy study in one direction of your choice to provide additional insight in the behaviour of 1 dimensional convolution (for eg, what happens with mixed signals, different file lengths, distorted filters, batch normalization etc). Provide a solid motivation and reasoning. 

#### OR

### Basic 2D convolution (implementation should  not directly use model.fit()

1. Create a database of 28*28 images with at least 1000 images. The database images each contain one circle, one triangle, or one rectangle. The triangles and rectangles can be all of the same oritentation, size, and shape but should have random locations. 
2. Design a simple convolutional network using convolution, pooling and other tools with a one-hot output to detect the shape of the objects. 
3. Test and optimize your setup with suitable methods. 
4. Extend this toy study in  one direction of your choice to provide additional insight in the behaviour of 2 dimensional convolution  (for eg, what happens with object distortions, varying object orientation, multiple objects, batch normalisation , etc). Provide solid motivation and reasoning. 

Replacing model.fit has typically been done as follows below 

In [ ]:

train_dataset = tf.data.Dataset.from_tensor_slices((x, y)) #x and y are training datasets to be defined beforehand
test_dataset = tf.data.Dataset.from_tensor_slices((xt, yt)) #xt and yt are test datasets to be defined beforehand

train_dataset = train_dataset.batch(50)
test_dataset = test_dataset.batch(25)

# optimizer and loss function to train
loss_object = tf.keras.losses.KLDivergence()  #Can experiment with this loss object accordingly to try to get better fits
optimizer = tf.keras.optimizers.Adam()

# Defining our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.KLDivergence('train_accuracy')   #If loss object is to be experimented with, would be ideal to match this with the above loss_object
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.KLDivergence('test_accuracy')    #If loss object is to be experimented with, would be ideal to match this with the above loss_object

def train_step(model, optimizer, x_train, y_train):
  with tf.GradientTape() as tape:
    predictions = model(x_train, training=True)
    loss = loss_object(y_train, predictions)
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  train_loss(loss)
  train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
  predictions = model(x_test)
  loss = loss_object(y_test, predictions)

  test_loss(loss)
  test_accuracy(y_test, predictions)

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

def my_modelf1():
  return tf.keras.models.Sequential([
    tf.keras.layers.Dense(2),
    tf.keras.layers.Dense(200, kernel_regularizer=tf.keras.regularizers.l2(1e-5),activation='relu'),
    tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(200, kernel_regularizer=tf.keras.regularizers.l2(1e-5),activation='relu'),
    tf.keras.layers.Dense(2,name="predictions", activation='linear')
  ])

In [ ]:
#running training on model as well as evaluating how well it fits on test data
modelf1 = my_modelf1()

EPOCHS = 20

for epoch in range(EPOCHS):
  for (x, y) in train_dataset:
    train_step(modelf1, optimizer, x, y)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (xt, yt) in test_dataset:
    test_step(modelf1, xt, yt)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
  
  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()